In [1]:
from __future__ import print_function, division,unicode_literals
import os
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
import librosa

from io import open
import unicodedata
import string
import re
import random

from tqdm.auto import tqdm
import json

import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import torch.nn as nn
from torch import optim
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from torch.utils.data.dataloader import default_collate


import torch.nn.functional as Fi


import torchaudio
import torchaudio.functional as F
import torchaudio.transforms as T

print(torch.__version__)
print(torchaudio.__version__)


import string

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

1.9.0
0.9.0


In [2]:
table_trans = str.maketrans(dict.fromkeys(string.punctuation))  # OR {key: None for key in string.punctuation}


path ="/Users/dami.osoba/work/bawk/small_dataset/small/CV_unpacked/cv-corpus-6.1-2020-12-11/en/validated.tsv"
meta = pd.read_csv(path,sep="\t")
meta_path = meta.set_index('path')

def read_manifest(path):
    manifest = []
    with open(path, 'r') as f:
        for line in tqdm(f, desc="Reading manifest data"):
            line = line.replace("\n", "")
            data = json.loads(line)
            manifest.append(data)
    return manifest
train_manifest_path = '/Users/dami.osoba/work/bawk/src/data/commonvoice_train_manifest.json'
train_manifest_data = read_manifest(train_manifest_path)
# keep audio < 4s
train_text = [data['text'] for data in train_manifest_data if data['duration']<=4]
train_path = [data['audio_filepath'] for data in train_manifest_data if data['duration']<=4]
train_path_pd = pd.DataFrame(train_path,columns=['train_path'])

# remove unicode
sentences = [c.encode(encoding="ascii",errors="ignore").decode().translate(table_trans) for c in train_text]
char_dict = sorted(list(set([b for a in sentences for b in a]))) +['EOS','SOS','PAD']
char_index = {a:char_dict.index(a) for a in char_dict}
dictOfindex = {char_dict.index(a):a for a in char_dict}
# char_index['EOS'] = len(char_dict)
# char_index['SOS'] = len(char_dict)+1
# char_index['PAD'] = len(char_dict)+2

Reading manifest data: 0it [00:00, ?it/s]

# Create voice dataset

In [4]:
class VoiceDataset(Dataset):
    def __init__(self, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
#         self.landmarks_frame = pd.read_csv(csv_file)
#         self.root_dir = root_dir
        self.transform = transform
        self.path_frame = train_path_pd

    def __len__(self):
        return len(self.path_frame)

    
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        waveform, _ = torchaudio.load(self.path_frame.loc[idx][0],)
        label = self.path_frame.loc[idx][0].split("/")[-1].split("wav")[0]+"mp3"
        # transcription for audio
        trans = meta_path.loc[label]['sentence']
        # encode to ascii
        trans = trans.encode(encoding="ascii",errors="ignore").decode().translate(table_trans).lower()
        chars =[b for a in trans for b in a]
        coded = [28]+[char_dict.index(a) for a in chars]+[27]

        sample = {'waveform': waveform, 'transcription': coded,'sentence':trans}

        if self.transform:
            sample = self.transform(sample)

        return sample

In [7]:
train_path[10]

'/Users/dami.osoba/work/bawk/src/data/small/train/wav/common_voice_en_22079780.wav'

In [17]:
def predict(wav_path,transcription):
    # Use the model to predict the label of the waveform
    waveform, _ = torchaudio.load(wav_path)
    sentence = transcription.encode(encoding="ascii",errors="ignore").decode().translate(table_trans)
    chars =[b for a in sentence for b in a]
    coded = [28,0]+[char_dict.index(a) for a in chars]+[0,27]
    sample['waveform'] =waveform
    sample['transcription'] = coded
    sample['sentence'] = sentence
    hmm = MelSpec()
    hah =hmm(sample)
    return hah

# Create FFT transform

In [ ]:
class augment(object):
    """Rescale the image in a sample to a given size.

    Args:
        output_size (tuple or int): Desired output size. If tuple, output is
            matched to output_size. If int, smaller of image edges is matched
            to output_size keeping aspect ratio the same.
    """
    def __init__(self):
        
        self.window_size = 25/1000
        self.stride = 10/1000
        self.sample_rate = 16000
        self.n_fft =int(self.window_size *self.sample_rate)
        self.win_length = None
        self.hop_length = int(self.sample_rate*self.stride)
        self.n_mels = 80
        self.max_time = 4
        pass
#         assert isinstance(output_size, (int, tuple))
#         self.output_size = output_size

    def mel_spectrogram(self,a):
        mel_spec = T.MelSpectrogram(
                        sample_rate=self.sample_rate,
                        n_fft=self.n_fft,
                        hop_length=self.hop_length,
                        center=True,
                        pad_mode="reflect",
                        power=2.0,
                        norm='slaney',
                        onesided=True,
                        n_mels=self.n_mels,
                        mel_scale="htk")
        return mel_spec(a)
        

    def __call__(self, sample):
        waveform, transcription,sentence = sample['waveform'], sample['transcription'],sample['sentence']
        #zero pad waveform
        zero_pad = torch.zeros(1, sample_rate*max_time- waveform.size()[1])
        padding = torch.cat([waveform,zero_pad],1)
        # get spectrogram
        wave_spec = self.mel_spectrogram(padding)
        wave_spec = wave_spec.swapaxes(1,2)
        #change transcription list to tensor
        transcription = torch.tensor(transcription, dtype=torch.long, device=device)


In [19]:
window_size = 25/1000
stride = 10/1000
sample_rate = 16000
n_fft =int(window_size *sample_rate)
win_length = None
hop_length = int(sample_rate*stride)
n_mels = 80
max_time = 4


mel_spectrogram = T.MelSpectrogram(
    sample_rate=sample_rate,
    n_fft=n_fft,
    hop_length=hop_length,
    center=True,
    pad_mode="reflect",
    power=2.0,
    norm='slaney',
    onesided=True,
    n_mels=n_mels,
    mel_scale="htk",
)

# melspec = mel_spectrogram(waveform)


class MelSpec(object):
    """Rescale the image in a sample to a given size.

    Args:
        output_size (tuple or int): Desired output size. If tuple, output is
            matched to output_size. If int, smaller of image edges is matched
            to output_size keeping aspect ratio the same.
    """
    def __init__(self):
        
        self.window_size = 25/1000
        self.stride = 10/1000
        self.sample_rate = 16000
        self.n_fft =int(self.window_size *self.sample_rate)
        self.win_length = None
        self.hop_length = int(self.sample_rate*self.stride)
        self.n_mels = 80
        self.max_time = 4
        pass
#         assert isinstance(output_size, (int, tuple))
#         self.output_size = output_size

    def mel_spectrogram(self,a):
        mel_spec = T.MelSpectrogram(
                        sample_rate=self.sample_rate,
                        n_fft=self.n_fft,
                        hop_length=self.hop_length,
                        center=True,
                        pad_mode="reflect",
                        power=2.0,
                        norm='slaney',
                        onesided=True,
                        n_mels=self.n_mels,
                        mel_scale="htk")
        return mel_spec(a)
        

    def __call__(self, sample):
        waveform, transcription,sentence = sample['waveform'], sample['transcription'],sample['sentence']
        #zero pad waveform
        zero_pad = torch.zeros(1, sample_rate*max_time- waveform.size()[1])
        padding = torch.cat([waveform,zero_pad],1)
        # get spectrogram
        wave_spec = self.mel_spectrogram(padding)
        wave_spec = wave_spec.swapaxes(1,2)
        #change transcription list to tensor
        transcription = torch.tensor(transcription, dtype=torch.long, device=device)

        return {'waveform': wave_spec, 'transcription': transcription, 'sentence':sentence}
    
transformed_dataset = VoiceDataset(transform = MelSpec())

for i in range(len(transformed_dataset)):
    sample = transformed_dataset[i]

    print(i, sample['waveform'].size(), sample['transcription'],sample['sentence'])

    if i == 3:
        break

0 torch.Size([1, 401, 80]) tensor([28, 20,  8,  1, 20,  0, 19, 15,  2,  5, 18,  5,  4,  0,  8,  9, 13,  0,
         1,  0, 12,  9, 20, 20, 12,  5, 27]) that sobered him a little
1 torch.Size([1, 401, 80]) tensor([28, 15, 16,  5, 14,  0,  3, 15, 14,  6,  5, 19, 19,  9, 15, 14,  0,  9,
        19,  0,  7, 15, 15,  4,  0,  6, 15, 18,  0, 20,  8,  5,  0, 19, 15, 21,
        12, 27]) open confession is good for the soul
2 torch.Size([1, 401, 80]) tensor([28,  2, 21, 20,  0, 20,  8,  5,  0,  5, 14,  7, 12,  9, 19,  8, 13,  1,
        14,  0, 23,  1, 19,  0,  5, 24, 21, 12, 20,  1, 14, 20, 27]) but the englishman was exultant
3 torch.Size([1, 401, 80]) tensor([28,  9,  0,  1, 13,  0,  6, 15, 12, 12, 15, 23,  9, 14,  7,  0, 13, 25,
         0,  4,  5, 19, 20,  9, 14, 25, 27]) i am following my destiny


In [24]:
dictOfchar

{' ': 0,
 'a': 1,
 'b': 2,
 'c': 3,
 'd': 4,
 'e': 5,
 'f': 6,
 'g': 7,
 'h': 8,
 'i': 9,
 'j': 10,
 'k': 11,
 'l': 12,
 'm': 13,
 'n': 14,
 'o': 15,
 'p': 16,
 'q': 17,
 'r': 18,
 's': 19,
 't': 20,
 'u': 21,
 'v': 22,
 'w': 23,
 'x': 24,
 'y': 25,
 'z': 26,
 'EOS': 27,
 'SOS': 28,
 'PAD': 29}

In [23]:
dictOfindex = { i : char_dict[i] for i in range(0, len(char_dict) ) }
dictOfchar = { char_dict[i]:i for i in range(0, len(char_dict) ) }

In [33]:
def pad_sequence(batch):
    # Make all tensor in a batch the same length by padding with zeros
    batch = [item.t() for item in batch]    
    batch = torch.nn.utils.rnn.pad_sequence(batch, batch_first=True, padding_value=0)
    return batch


def pad_collate(batch):
    max_input_len = float('-inf')
    max_target_len = float('-inf')

    for elem in batch:
        feature = elem['waveform']
        feature = feature.squeeze()
        trn = elem['transcription']
        max_input_len = max_input_len if max_input_len > feature.shape[0] else feature.shape[0]
        max_target_len = max_target_len if max_target_len > len(trn) else len(trn)

    for i, elem in enumerate(batch):
        f = elem['waveform']
        trn = elem['transcription']
        sentence = elem['sentence']
        f = f.squeeze()
        input_length = f.shape[0]
        input_dim = f.shape[1]
        print(max_input_len)
        # print('f.shape: ' + str(f.shape))
        feature = np.zeros((max_input_len, input_dim), dtype=np.float32)
        feature[:f.shape[0], :f.shape[1]] = f
        trn = np.pad(trn, (0, max_target_len - len(trn)), 'constant', constant_values=29)
        batch[i] = (feature, trn, input_length,sentence)
        # print('feature.shape: ' + str(feature.shape))
        # print('trn.shape: ' + str(trn.shape))

    batch.sort(key=lambda x: x[2], reverse=True)

    return default_collate(batch)



def collate_fn(batch):

    # A data tuple has the form:
    # waveform, sample_rate, label, speaker_id, utterance_number

    tensors, targets,sentence = [], [],[]

    # Gather in lists, and encode labels as indices
    for a in batch:
        tensors += [a['waveform']]
        targets += [a['transcription']]
        sentence += [a['sentence']]
                   
    # Group the list of tensors into a batched tensor
    tensors = tensors
#     targets = torch.stack(targets)
    targets = pad_sequence(targets)

    return tensors, targets,sentence


train_loader = DataLoader(transformed_dataset, batch_size=10,collate_fn=pad_collate,
                        shuffle=True, num_workers=0)

iterator = iter(train_loader)
x_batch,y,input_lengths,sentence = iterator.next()
print(x_batch,y,input_lengths,sentence)

389
389
389
389
389
389
389
389
389
389
tensor([[[0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         [2.1369e-12, 7.4987e-12, 4.5659e-12,  ..., 7.5317e-13,
          6.2066e-13, 3.9604e-13],
         [4.5014e-08, 1.5796e-07, 3.8889e-08,  ..., 2.6783e-10,
          2.6160e-10, 1.7072e-10],
         ...,
         [2.8880e-08, 1.0134e-07, 8.3834e-08,  ..., 1.1125e-09,
          1.0242e-09, 2.0016e-10],
         [8.1334e-07, 2.8541e-06, 7.7896e-07,  ..., 5.5402e-10,
          7.7795e-10, 2.1970e-10],
         [1.5339e-08, 5.3828e-08, 3.4000e-08,  ..., 3.2020e-10,
          4.7531e-10, 5.3748e-10]],

        [[0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         [5.6637e-09, 1.9875e-08, 1.1874e-08,  ..., 1.1692e-11,
          2.9177e-11, 9.1143e-11],
         ...,
         [1.5303e-07, 5.3702e-07, 3.786

In [30]:
x_batch.shape

torch.Size([10, 385, 80])

In [16]:
d1 = transformed_dataset[10]['waveform']

In [18]:
d1.shape

torch.Size([1, 363, 80])

In [36]:
n1 = nn.GRU(80,10)
n2 = nn.GRU(10, 10)
n3 = nn.GRU(10, 10)

In [37]:
o1,o2 = n1(x_batch)
b1,b2 = n2(o1)

In [44]:
hmm = nn.GRU(80,10,)
input_x = x_batch.size(1)
enc_len = x_batch.size(2)
total_length = x_batch.size(1)
packed_input = pack_padded_sequence(x_batch, input_lengths, batch_first=True)
hah, _= hmm(packed_input)
output, _ = pad_packed_sequence(hah, batch_first=True, total_length=total_length)

In [84]:
ys = [y[y != 29] for y in padded_input]  # parse padded ys
# prepare input and output word sequences with sos/eos IDs
eos = ys[0].new([27])
sos = ys[0].new([28])
ys_in = [y for y in ys]
ys_out = [y for y in ys]
# padding for ys with -1
# pys: utt x olen
ys_in_pad = pad_list(ys_in, 27)
ys_out_pad = pad_list(ys_out, 29)
assert ys_in_pad.size() == ys_out_pad.size()
batch_size = ys_in_pad.size(0)
output_length = ys_in_pad.size(1)

In [ ]:
class Encoder(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, dropout=0.0, bidirectional=True):
        super(Encoder, self).__init__()
        self.rnn = nn.GRU(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout,
                          bidirectional=bidirectional)

    def forward(self, input_x, enc_len):
        total_length = input_x.size(1)  # get the max sequence length
        # print('total_length: ' + str(total_length))
        # print('input_x.size(): ' + str(input_x.size()))
        packed_input = pack_padded_sequence(input_x, enc_len, batch_first=True)
        # print('enc_len: ' + str(enc_len))
        packed_output, hidden = self.rnn(packed_input)
        output, _ = pad_packed_sequence(packed_output, batch_first=True, total_length=total_length)
        return output, hidden
    
hmm = nn.GRU(80,10,)
input_x = x_batch.size(1)
enc_len = x_batch.size(2)
total_length =x_batch.size(1)
packed_input = pack_padded_sequence(x_batch, input_lengths, batch_first=True)
hah, _= hmm(packed_input)
output, _ = pad_packed_sequence(hah, batch_first=True, total_length=total_length)


In [ ]:
class AttnDecoder(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input)
#         embedded ,_= self.embedding(input)

        embedded = self.dropout(embedded)

        attn_weights = Fi.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = Fi.relu(output)
        output, hidden = self.gru(output, hidden)

        output = Fi.log_softmax(self.out(output[0]), dim=1)
        output_probs = torch.exp(output)
        return output, hidden, attn_weights,output_probs

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [20]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

#         self.embedding = nn.Embedding(input_size, hidden_size,)
        self.embedding = nn.GRU(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden,total_length):
        output,_ = self.embedding(input)
        
        
        output, hidden = self.gru(output, hidden)
#         m = nn.MaxPool1d(MAX_LENGTH)
#         yy = m(hidden.swapaxes(1,2))
#         yy = yy.swapaxes(1,2)
        return output, hidden

    def initHidden(self):
#         return torch.zeros(1, MAX_LENGTH, self.hidden_size, device=device)
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [ ]:
class EncoderTrue(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

#         self.embedding = nn.Embedding(input_size, hidden_size,)
        self.embedding = nn.GRU(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden,total_length):
        output,_ = self.embedding(input)      
        output, hidden = self.gru(output, hidden)

        return output, hidden

    def initHidden(self):
        return torch.zeros(1, MAX_LENGTH, self.hidden_size, device=device)


In [21]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
#         output = self.embedding(input)
        print(output.shape)
        output = Fi.relu(output)
        print(hidden.shape,output.shape)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1,1, self.hidden_size, device=device)

In [29]:
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
#         embedded ,_= self.embedding(input)

        embedded = self.dropout(embedded)

        attn_weights = Fi.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = Fi.relu(output)
        output, hidden = self.gru(output, hidden)

        output = Fi.log_softmax(self.out(output[0]), dim=1)
        output_probs = torch.exp(output)
        return output, hidden, attn_weights,output_probs

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [30]:
teacher_forcing_ratio = 0.2
MAX_LENGTH = 401
SOS_token = 28
EOS_token = 27


def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer,
          decoder_optimizer, criterion,total_length, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(1)
    tot_length = total_length
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

#     for ei in range(input_length):
#         encoder_output, encoder_hidden = encoder(
#             input_tensor[:,:,ei].view(1,1,80), encoder_hidden)
#         encoder_outputs[ei] = encoder_output[0, 0]

#     encoder_output, encoder_hidden = encoder(input_tensor.reshape(1,MAX_LENGTH,80), encoder_hidden)

    encoder_output, encoder_hidden = encoder(input_tensor, encoder_hidden,tot_length)

    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention,output_probs = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention,output_probs = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.topk(1)
#             yay = torch.distributions.categorical.Categorical(output_probs)
#             topi = yay.sample()
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [31]:
teacher_forcing_ratio = 0.80
MAX_LENGTH = 401
SOS_token = 28
EOS_token = 27


def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer,
          decoder_optimizer, criterion,total_length, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(1)
    tot_length = total_length
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

#     for ei in range(input_length):
#         encoder_output, encoder_hidden = encoder(
#             input_tensor[:,:,ei].view(1,1,80), encoder_hidden)
#         encoder_outputs[ei] = encoder_output[0, 0]

#     encoder_output, encoder_hidden = encoder(input_tensor.reshape(1,MAX_LENGTH,80), encoder_hidden)

    encoder_output, encoder_hidden = encoder(input_tensor, encoder_hidden,tot_length)

    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention,output_probs = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention,output_probs = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.topk(1)
            yay = torch.distributions.categorical.Categorical(output_probs)
            topi = yay.sample()
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [24]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [25]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np


def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)


def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every
    
    lns = len(transformed_dataset)

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_examples = np.random.choice(lns-1,n_iters)
                      
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters):
        training_pair = transformed_dataset[training_examples[iter-1]]
        input_tensor = training_pair['waveform'].reshape(1,1,80*401)
        target_tensor = torch.tensor(training_pair['transcription'], dtype=torch.long, device=device).view(-1, 1) 
        tot = input_tensor.size(1)

        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion,tot)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / (plot_every*1.0)
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)
    
    


# Train model

In [32]:
hidden_size = 20
encoder1 = EncoderRNN(80*401, hidden_size).to(device)
attn_decoder2 = DecoderRNN(hidden_size, 29).to(device)

attn_decoder1 = AttnDecoderRNN(hidden_size, 29, dropout_p=0.1).to(device)

trainIters(encoder1, attn_decoder1, 5000, print_every=1000, plot_every=2000,learning_rate=0.01)

/Users/dami.osoba/opt/anaconda3/envs/tf/lib/python3.7/site-packages/ipykernel_launcher.py:33: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


0m 29s (- 1m 59s) (1000 20%) 2.4123
0m 58s (- 1m 27s) (2000 40%) 2.2728
1m 25s (- 0m 57s) (3000 60%) 2.1818
1m 54s (- 0m 28s) (4000 80%) 2.1457


In [36]:
trainIters(encoder1, attn_decoder1, 10000, print_every=1000, plot_every=2000,learning_rate=0.01)

/Users/dami.osoba/opt/anaconda3/envs/tf/lib/python3.7/site-packages/ipykernel_launcher.py:33: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


0m 28s (- 4m 20s) (1000 10%) 2.0946
0m 59s (- 3m 56s) (2000 20%) 2.0657
1m 28s (- 3m 27s) (3000 30%) 2.0727
1m 57s (- 2m 55s) (4000 40%) 2.0609
2m 26s (- 2m 26s) (5000 50%) 2.0230
2m 56s (- 1m 57s) (6000 60%) 2.0583
3m 28s (- 1m 29s) (7000 70%) 2.0091
4m 1s (- 1m 0s) (8000 80%) 1.9688
4m 34s (- 0m 30s) (9000 90%) 1.9660


In [46]:
PATH = "/Users/dami.osoba/work/bawk/models/dec_model"
torch.save(attn_decoder1, PATH)

In [47]:
PATH = "/Users/dami.osoba/work/bawk/models/enc_model"
torch.save(encoder1, PATH)

In [50]:
enmodel = torch.load("/Users/dami.osoba/work/bawk/models/enc_model")
enmodel.eval()

decmodel = torch.load("/Users/dami.osoba/work/bawk/models/dec_model")
decmodel.eval()

AttnDecoderRNN(
  (embedding): Embedding(29, 20)
  (attn): Linear(in_features=40, out_features=401, bias=True)
  (attn_combine): Linear(in_features=40, out_features=20, bias=True)
  (dropout): Dropout(p=0.1, inplace=False)
  (gru): GRU(20, 20)
  (out): Linear(in_features=20, out_features=29, bias=True)
)

In [ ]:
def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        choice = np.random.randint(200)
        print(choice)
        actual = transformed_dataset[choice]['sentence']
        ex = transformed_dataset[choice]['waveform']
        output_words, attentions,_ = evaluate(encoder, decoder, ex)
        output_sentence = ''.join(output_words)
        print(actual, '<', output_sentence)
        print('')

In [78]:
train_path[23]

'/Users/dami.osoba/work/bawk/src/data/small/train/wav/common_voice_en_119561.wav'

In [79]:
train_text[23]

'that might be all right.'

In [77]:
predict(train_path[23],train_text[23],enmodel,decmodel)

that might be all right < SOSthe dige tone<EOS>



In [76]:
def predict(wav_path,transcription,encoder,decoder):
    # Use the model to predict the label of the waveform
    waveform, _ = torchaudio.load(wav_path)
    sentence = transcription.encode(encoding="ascii",errors="ignore").decode().translate(table_trans)
    chars =[b for a in sentence for b in a]
    coded = [28]+[char_dict.index(a) for a in chars]+[27]
    sample['waveform'] =waveform
    sample['transcription'] = coded
    sample['sentence'] = sentence
    transformer = MelSpec()
    mels =transformer(sample)
    ex =mels['waveform']
    
    output_words, attentions,_ = evaluate(encoder, decoder, ex)
    output_sentence = ''.join(output_words)
    return output_sentence
    

In [67]:
def evaluate(encoder, decoder, tens, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tens
        input_length = input_tensor.size(2)
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        encoder_output, encoder_hidden = encoder(input_tensor.reshape(1,1,80*401), encoder_hidden,MAX_LENGTH)


        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden
        decoder_output = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):
            decoder_output, decoder_hidden, decoder_attention,decoder_probs = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
            yay = torch.distributions.categorical.Categorical(decoder_probs)
            topi = yay.sample()
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(dictOfindex[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words, decoder_attentions[:di + 1],decoder_output

In [34]:
def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        choice = np.random.randint(200)
        print(choice)
        actual = transformed_dataset[choice]['sentence']
        ex = transformed_dataset[choice]['waveform']
        output_words, attentions,_ = evaluate(encoder, decoder, ex)
        output_sentence = ''.join(output_words)
        print(actual, '<', output_sentence)
        print('')

# Really bad model

In [39]:
evaluateRandomly(encoder1, attn_decoder1)


168
you said it to me too < SOSthet hy one ill ster a troulenpf<EOS>

183
now look what youve done < SOScoof honing ipler<EOS>

13
two < SOSyo lrlieb cowiovere zoon<EOS>

62
he disappeared into the tent < SOShegiver a of a the marze her erpeesen up eaniap browndoyrewid for adyorcrawntent witt dearntnged on u<EOS>

166
ill talk to you tonight < SOSyou poet fee hy odous courldengtas i sotellndingty on feny ontouenilhan<EOS>

27
how did it happen < SOSthee loong uperee right spigel ven<EOS>

157
richard has gone camping by himself < SOShe it fakrieve auntpy a seaits salingedirs t neees feccid<EOS>

170
what shall we do now < SOSwhot mir they inrey<EOS>

152
good weekend is edited by amelia lester < SOSjex do ge goes lhth<EOS>

11
say what you have got to say < SOSser<EOS>

